# MSOA Data Preprocessing

The objective is to obtain the MSOA data with the following columns:
- Vaccine uptake rates;
- IMD Decile: 1-10;
- Deprivation domain scores (7 domains, original score): these scores are aggregated from LSOAs.
- Ethnic composition (asian, white, black, mixed, other);
- pct_hh_car: percentage of households with at least one car or van

Note that London MSOAs are included in this data.

In [45]:
# read the data and take a look at the shape
import pandas as pd
import geopandas as gpd
import os
import numpy as np

## load data

### number of people getting vaccinated
source:
https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-vaccinations/covid-19-vaccinations-archive/

In [46]:
file_vaccine_uptake = 'vaccination_pop_2021_11_18.csv'
df_vaccine_uptake = pd.read_csv(file_vaccine_uptake, low_memory=False)
# why dropping these MSOAs? The vaccination population of these two MSOAs are combined
df_vaccine_uptake=df_vaccine_uptake.drop(df_vaccine_uptake[df_vaccine_uptake.MSOA=='E02000001/E02000371'].index)
df_vaccine_uptake=df_vaccine_uptake.drop(df_vaccine_uptake[df_vaccine_uptake.MSOA=='E02003950/E02006781'].index)
df_vaccine_uptake

,MSOA,18over1st_dose,18over2nd_dose
0,E02002796,4810,4712
1,E02002797,5906,5796
2,E02002798,7919,7730
3,E02002799,4234,4036
4,E02002800,5209,5057
...,...,...,...
6784,E02002478,4317,4134
6785,E02002479,5990,5748
6786,E02002480,4001,3812
6787,E02002481,5043,4803


In [47]:
df_pop[df_pop.MSOA.isin(['E02000001','E02000371','E02003950','E02006781'])]

,MSOA,pop0_17,pop18over
1385,E02003950,1032,4556
1400,E02006781,339,1887
5808,E02000001,1825,9113
6178,E02000371,2017,14892


In [48]:
# read population data (UK). Merge and compute uptake rates
url = 'population_msoa_18over.csv'
df_pop = pd.read_csv(url, low_memory=False)
print(f"Data frame is {df_pop.shape[0]:,} x {df_pop.shape[1]}")

Data frame is 7,201 x 3


In [49]:
# merge two dataset
df_vaccine_uptake = pd.merge(left=df_vaccine_uptake, right=df_pop, how='left', left_on='MSOA', right_on='MSOA')
print(f"Data frame is {df_vaccine_uptake.shape[0]:,} x {df_vaccine_uptake.shape[1]}")
# MSOA.drop(columns=['MSOA','OBJECTID'], axis=1,inplace=True)

Data frame is 6,787 x 5


In [50]:
# compute vaccine uptake rates
df_vaccine_uptake['vaccination_percentage_1stdose'] = df_vaccine_uptake['18over1st_dose']/df_vaccine_uptake['pop18over']
df_vaccine_uptake['vaccination_percentage_2nddose'] = df_vaccine_uptake['18over2nd_dose']/df_vaccine_uptake['pop18over']
df_vaccine_uptake['vaccination_percentage_total'] = (df_vaccine_uptake['18over1st_dose']+df_vaccine_uptake['18over2nd_dose'])/df_vaccine_uptake['pop18over']/2

# Set vaccination rate greater than 1 to 1
df_vaccine_uptake['vaccination_percentage_2nddose']= np.where(df_vaccine_uptake['vaccination_percentage_2nddose']>1, 1, df_vaccine_uptake['vaccination_percentage_2nddose'])
df_vaccine_uptake['vaccination_percentage_1stdose']= np.where(df_vaccine_uptake['vaccination_percentage_1stdose']>1, 1, df_vaccine_uptake['vaccination_percentage_1stdose'])
df_vaccine_uptake['vaccination_percentage_total']= np.where(df_vaccine_uptake['vaccination_percentage_total']>1, 1, df_vaccine_uptake['vaccination_percentage_total'])
print(df_vaccine_uptake.columns)

Index(['MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17', 'pop18over',
       'vaccination_percentage_1stdose', 'vaccination_percentage_2nddose',
       'vaccination_percentage_total'],
      dtype='object')


In [51]:
print(df_vaccine_uptake.shape)

(6787, 8)


## Add MSOA IMD decile/ethnic/car

In [54]:
# read MSOA IMD decile file
# Read the IMD data
imd = pd.read_csv('imd2019_msoa_level_data.csv')
# Read the Ethnic group data
ethnic = pd.read_csv('ethnic_group_2011.csv')
# read car data
car = pd.read_csv('CARVAN_MSOAIZ_England_Scotland_Wales_Descriptions.csv')
# Merge IMD/Ethnic/Car data
df_vaccine_uptake = pd.merge(left=df_vaccine_uptake, right=imd[['MSOAC','IMD19 SCORE','MSOADECILE']], how='left', left_on='MSOA', right_on='MSOAC')
df_vaccine_uptake = pd.merge(left=df_vaccine_uptake, right=ethnic, how='left', left_on='MSOA', right_on='MSOA')
df_vaccine_uptake = pd.merge(left=df_vaccine_uptake, right=car, how='left', left_on='MSOA', right_on='GEO_CODE')

In [55]:
print(df_vaccine_uptake.columns)

Index(['MSOA', '18over1st_dose', '18over2nd_dose', 'pop0_17', 'pop18over',
       'vaccination_percentage_1stdose', 'vaccination_percentage_2nddose',
       'vaccination_percentage_total', 'MSOAC', 'IMD19 SCORE', 'MSOADECILE',
       'White%', 'Mixed%', 'Asian%', 'Black%', 'Other%', 'GEO_CODE',
       'Total_households', 'Households_with_at_least_one_vars_or_vans',
       'Per_cent_of_households_with_at_least_one_car_or_van'],
      dtype='object')


In [56]:
# Drop useless columns
df_vaccine_uptake = df_vaccine_uptake.drop(['MSOAC', 'Households_with_at_least_one_vars_or_vans','Total_households','GEO_CODE'], axis=1)

## Calculate MSOA-level deprivation domain scores from LSOA data

In [68]:
df_lsoa_msoa = pd.read_csv("LSOA_MSOA.csv").drop_duplicates()
print(df_lsoa_msoa.shape)
print(df_lsoa_msoa.head())
print(df_lsoa_msoa.MSOA11CD.nunique())
print("Are all England MSOAs in this table?")
print(df_vaccine_uptake.MSOA.isin(df_lsoa_msoa.MSOA11CD).all())
# print(df_vaccine_uptake.MSOA[-df_vaccine_uptake.MSOA.isin(df_lsoa_msoa.MSOA11CD)])
# print('E02000001' in df_lsoa_msoa.MSOA11CD.unique())
# print('E02000371' in df_lsoa_msoa.MSOA11CD.unique())
# print('E02003950' in df_lsoa_msoa.MSOA11CD.unique())
# print('E02006781' in df_lsoa_msoa.MSOA11CD.unique())

(41729, 2)
    LSOA11CD   MSOA11CD
0  E01011966  E02002488
1  E01011974  E02002487
2  E01011965  E02002488
3  E01011983  E02002488
4  E01011950  E02002490
8480
Are all England MSOAs in this table?
True


In [110]:
# read LSOA deprivation data
df_imd_lsoa = pd.read_csv("File_7_-_All_IoD2019_Scores__Ranks__Deciles_and_Population_Denominators_3.csv")
# print(df_imd_lsoa.columns)

In [111]:
# keep useful columns
df_imd_lsoa = df_imd_lsoa[['LSOA code (2011)',
                          'Income Score (rate)',
                         'Employment Score (rate)',
                         'Education, Skills and Training Score',
                         'Health Deprivation and Disability Score',
                          'Crime Score',
                          'Barriers to Housing and Services Score',
                          'Living Environment Score',
                           'Total population: mid 2015 (excluding prisoners)'
                         ]]
print(df_imd_lsoa.shape)

(32844, 9)


In [112]:
# add MSOA
df_imd_lsoa = pd.merge(left=df_imd_lsoa, right=df_lsoa_msoa, how='left', left_on='LSOA code (2011)', right_on='LSOA11CD')
# print(df_imd_lsoa[df_imd_lsoa.MSOA11CD.isna()])

In [113]:
print(df_imd_lsoa.shape)
# print(df_imd_lsoa)

(32844, 11)


In [114]:
df_imd_lsoa.columns

Index(['LSOA code (2011)', 'Income Score (rate)', 'Employment Score (rate)',
       'Education, Skills and Training Score',
       'Health Deprivation and Disability Score', 'Crime Score',
       'Barriers to Housing and Services Score', 'Living Environment Score',
       'Total population: mid 2015 (excluding prisoners)', 'LSOA11CD',
       'MSOA11CD'],
      dtype='object')

In [117]:
df_imd_lsoa = df_imd_lsoa.rename(columns={"Total population: mid 2015 (excluding prisoners)": "pop"})
wm = lambda x: np.average(x, weights=df_imd_lsoa.loc[x.index, "pop"])

# groupby MSOAs and aggregate by weighted mean on the domain scores
# Define a lambda function to compute the weighted mean:

df_imd_msoa = df_imd_lsoa.groupby('MSOA11CD').agg(income_score=("Income Score (rate)", wm),  
                                                  employ_score=("Employment Score (rate)", wm),
                                                  edu_score=("Education, Skills and Training Score", wm),
                                                  health_score=("Health Deprivation and Disability Score", wm),
                                                  crime_score=("Crime Score", wm),
                                                  housing_score=("Barriers to Housing and Services Score", wm),
                                                  livEnv_score=("Living Environment Score", wm),
                                                 )

In [120]:
df_imd_msoa = df_imd_msoa.reset_index()

In [121]:
# merge with df_vaccine_uptake
df_vaccine_uptake = pd.merge(left=df_vaccine_uptake, right=df_imd_msoa, how='left', left_on='MSOA', right_on='MSOA11CD')

In [129]:
df_vaccine_uptake = df_vaccine_uptake.rename(columns={'MSOADECILE':'msoa_imd_decile', 'White%':'pct_White', 'Mixed%':'pct_Mixed',
                                                     'Asian%':'pct_Asian', 'Black%':'pct_black','Other%':'pct_other',
                                                      'Per_cent_of_households_with_at_least_one_car_or_van':'pct_hh_car'})

In [130]:
df_vaccine_uptake.to_csv('vaccine_uptake_socioeco.csv')

In [128]:
## Do not run the cells below

### read population data (mid2020 csv)

source:
https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/middlesuperoutputareamidyearpopulationestimates

In [3]:
url = 'population_msoa_18over.csv'
df = pd.read_csv(url, low_memory=False)
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 7,201 x 3


In [4]:
df

,MSOA,pop0_17,pop18over
0,E02002483,2362,7970
1,E02002484,2400,8040
2,E02002485,1987,6178
3,E02002487,934,4240
4,E02002488,1145,4749
...,...,...,...
7196,W02000286,1463,5505
7197,W02000287,1714,6784
7198,W02000288,1636,6465
7199,W02000289,1812,6973


### read MSOA boundary data (and exclude London region)

source:
https://geoportal.statistics.gov.uk/datasets/ons::middle-layer-super-output-areas-december-2011-boundaries-super-generalised-clipped-bsc-ew-v3/about

In [3]:
msoa = gpd.read_file('MSOA_boundary.gpkg')
msoa = msoa.to_crs('epsg:27700')

drop useless columns

In [4]:
drop_column = ['MSOA11NMW','BNG_E','BNG_N','LONG','LAT','Shape__Length','Shape__Area']
msoa.drop(columns=drop_column, axis=1,inplace=True)

filter the data, only leave england data

In [5]:
msoa = msoa[msoa.MSOA11CD.str.startswith('E')]
msoa.shape

(6791, 4)

Drop rows that are have problem

In [45]:
msoa=msoa.drop(msoa[msoa.MSOA11CD=='E02000001'].index)
msoa=msoa.drop(msoa[msoa.MSOA11CD=='E02000371'].index)
msoa=msoa.drop(msoa[msoa.MSOA11CD=='E02003950'].index)
msoa=msoa.drop(msoa[msoa.MSOA11CD=='E02006781'].index)

join population data

In [46]:
MSOA = pd.merge(left=msoa, right=df, how='left', left_on='MSOA11CD', right_on='MSOA')
MSOA.drop(columns=['MSOA','OBJECTID'], axis=1,inplace=True)

Exclude London region

In [47]:
LA_MSOA = pd.read_csv('LA_MSOA_code.csv', low_memory=False)
aaa = pd.merge(left=MSOA, right=LA_MSOA, how='left', left_on='MSOA11CD', right_on='MSOA Code')
aaa.drop(columns=['LA_name_2020','MSOA Code'], axis=1,inplace=True)

In [48]:
London = gpd.read_file('London_Boroughs.gpkg')
London = London.to_crs('epsg:27700')
London.drop(columns=['objectid','name','hectares','nonld_area','ons_inner','sub_2011','geometry'], axis=1,inplace=True)

In [49]:
aaa = pd.merge(left=aaa, right=London, how='left', left_on='LA_Code_2020', right_on='gss_code')
MSOA_excludeLondon=aaa[aaa.isnull().T.any()].reset_index()
MSOA_excludeLondon.drop(columns=['index','LA_Code_2020','gss_code'], axis=1,inplace=True)
MSOA_excludeLondon

,MSOA11CD,MSOA11NM,geometry,pop0_17,pop18over
0,E02000984,Bolton 001,"POLYGON ((372121.741 414318.582, 372147.184 41...",1462,5998
1,E02000985,Bolton 002,"POLYGON ((372971.325 411456.076, 373104.966 41...",1328,5968
2,E02000986,Bolton 003,"POLYGON ((372147.184 413616.095, 372503.949 41...",1714,6758
3,E02000987,Bolton 004,"POLYGON ((363078.556 411480.529, 363041.920 41...",1251,5891
4,E02000988,Bolton 005,"POLYGON ((371044.136 412457.779, 371567.570 41...",2176,6749
...,...,...,...,...,...
5801,E02006922,Colchester 022,"MULTIPOLYGON (((601481.105 224815.326, 601485....",2699,11342
5802,E02006926,Thurrock 020,"POLYGON ((561100.600 178929.667, 560928.979 17...",2342,5982
5803,E02006932,Liverpool 060,"POLYGON ((335757.632 390987.474, 335739.219 39...",511,18899
5804,E02006933,Liverpool 061,"POLYGON ((335096.788 389638.891, 334715.024 38...",654,8174


### read MSOA population weighted centroids data

set population weighted centroids as demand point

source:
https://geoportal.statistics.gov.uk/datasets/ons::middle-layer-super-output-areas-december-2011-population-weighted-centroids/about

In [13]:
demand = gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_raw/MSOA_Population_Weighted_Centroids.gpkg?raw=true')
demand = demand.to_crs('epsg:27700')

filter the data, only leave england data

In [14]:
demand = demand[demand.msoa11cd.str.startswith('E')]
demand.shape

(6791, 4)

In [15]:
demand = pd.merge(left=demand, right=df, how='left', left_on='msoa11cd', right_on='MSOA')
demand.drop(columns=['MSOA','msoa11nm'], axis=1,inplace=True)

In [16]:
demand

,objectid,msoa11cd,geometry,pop0_17,pop18over
0,1,E02002536,POINT (445582.345 524175.434),2171,7556
1,2,E02002537,POINT (446777.151 524256.841),2286,6293
2,3,E02002534,POINT (461356.929 515118.900),833,4806
3,4,E02002535,POINT (446117.027 525455.836),1640,7615
4,5,E02002532,POINT (461053.212 516175.379),1248,5559
...,...,...,...,...,...
6786,6787,E02004669,POINT (393469.114 227500.260),1468,6161
6787,6788,E02006096,POINT (332829.367 109219.836),1451,5570
6788,6789,E02003088,POINT (343276.340 158947.520),1706,6910
6789,6790,E02006070,POINT (331710.269 136880.200),1893,6627


## read vaccination site point data, clean and reformat

In [33]:
hospital = gpd.read_file('site_final/final/hospital_hub.gpkg')
hospital = hospital.to_crs('epsg:27700')

pharmacy = gpd.read_file('site_final/final/pharmacy.gpkg')
pharmacy = pharmacy.to_crs('epsg:27700')

GP = gpd.read_file('site_final/final/GP.gpkg')
GP = GP.to_crs('epsg:27700')

vc = gpd.read_file('site_final/final/vaccination_center.gpkg')
vc = vc.to_crs('epsg:27700')

In [34]:
# Drop useless columns
hospital=hospital[['geometry']]
pharmacy=pharmacy[['geometry']]
GP=GP[['geometry']]
vc=vc[['geometry']]

In [35]:
print("Hospital Hub number:",hospital.shape[0])
print("GP:",GP.shape[0])
print("Pharmacy:",pharmacy.shape[0])
print("Vaccination Center number:",vc.shape[0])
print("Total:",vc.shape[0]+pharmacy.shape[0]+hospital.shape[0]+GP.shape[0])

Hospital Hub number: 228
GP: 1055
Pharmacy: 1446
Vaccination Center number: 105
Total: 2834


In [36]:
# concat four types of vaccination site data into one dataframe
site = pd.concat([hospital, GP, pharmacy, vc], axis=0, ignore_index=True).reset_index()

In [37]:
site['supply_value']=1
site

,index,geometry,supply_value
0,0,POINT (527671.845 409625.582),1
1,1,POINT (532582.907 182659.957),1
2,2,POINT (433880.091 387238.877),1
3,3,POINT (546372.813 254986.453),1
4,4,POINT (338131.255 397000.897),1
...,...,...,...
2829,2829,POINT (301789.330 89707.897),1
2830,2830,POINT (361632.845 178155.149),1
2831,2831,POINT (297993.791 145504.134),1
2832,2832,POINT (166574.610 27864.985),1


## Export the data

In [38]:
site.to_file("vaccination_site.gpkg", driver="GPKG")

In [51]:
MSOA_excludeLondon.to_file("MSOA_Boundary_with_population_excludeLondon.gpkg", driver="GPKG")

In [30]:
demand.to_file("MSOA_Population_Weighted_Centroids_with_population.gpkg", driver="GPKG")

In [50]:
MSOA_excludeLondon

,MSOA11CD,MSOA11NM,geometry,pop0_17,pop18over
0,E02000984,Bolton 001,"POLYGON ((372121.741 414318.582, 372147.184 41...",1462,5998
1,E02000985,Bolton 002,"POLYGON ((372971.325 411456.076, 373104.966 41...",1328,5968
2,E02000986,Bolton 003,"POLYGON ((372147.184 413616.095, 372503.949 41...",1714,6758
3,E02000987,Bolton 004,"POLYGON ((363078.556 411480.529, 363041.920 41...",1251,5891
4,E02000988,Bolton 005,"POLYGON ((371044.136 412457.779, 371567.570 41...",2176,6749
...,...,...,...,...,...
5801,E02006922,Colchester 022,"MULTIPOLYGON (((601481.105 224815.326, 601485....",2699,11342
5802,E02006926,Thurrock 020,"POLYGON ((561100.600 178929.667, 560928.979 17...",2342,5982
5803,E02006932,Liverpool 060,"POLYGON ((335757.632 390987.474, 335739.219 39...",511,18899
5804,E02006933,Liverpool 061,"POLYGON ((335096.788 389638.891, 334715.024 38...",654,8174


In [32]:
demand

,objectid,msoa11cd,geometry,pop0_17,pop18over
0,1,E02002536,POINT (445582.345 524175.434),2171,7556
1,2,E02002537,POINT (446777.151 524256.841),2286,6293
2,3,E02002534,POINT (461356.929 515118.900),833,4806
3,4,E02002535,POINT (446117.027 525455.836),1640,7615
4,5,E02002532,POINT (461053.212 516175.379),1248,5559
...,...,...,...,...,...
6786,6787,E02004669,POINT (393469.114 227500.260),1468,6161
6787,6788,E02006096,POINT (332829.367 109219.836),1451,5570
6788,6789,E02003088,POINT (343276.340 158947.520),1706,6910
6789,6790,E02006070,POINT (331710.269 136880.200),1893,6627


In [8]:
MSOA = pd.merge(left=msoa, right=df, how='left', left_on='MSOA11CD', right_on='MSOA')
MSOA.drop(columns=['MSOA','OBJECTID'], axis=1,inplace=True)
LA_MSOA = pd.read_csv('LA_MSOA_code.csv', low_memory=False)
aaa = pd.merge(left=MSOA, right=LA_MSOA, how='left', left_on='MSOA11CD', right_on='MSOA Code')
aaa.drop(columns=['LA_name_2020','MSOA Code'], axis=1,inplace=True)
London = gpd.read_file('London_Boroughs.gpkg')
London = London.to_crs('epsg:27700')
London.drop(columns=['objectid','name','hectares','nonld_area','ons_inner','sub_2011','geometry'], axis=1,inplace=True)
aaa = pd.merge(left=aaa, right=London, how='left', left_on='LA_Code_2020', right_on='gss_code')
MSOA_excludeLondon_base=aaa[aaa.isnull().T.any()].reset_index()
MSOA_excludeLondon_base.drop(columns=['index','LA_Code_2020','gss_code'], axis=1,inplace=True)
MSOA_excludeLondon_base

,MSOA11CD,MSOA11NM,geometry,pop0_17,pop18over
0,E02000984,Bolton 001,"POLYGON ((372121.741 414318.582, 372147.184 41...",1462,5998
1,E02000985,Bolton 002,"POLYGON ((372971.325 411456.076, 373104.966 41...",1328,5968
2,E02000986,Bolton 003,"POLYGON ((372147.184 413616.095, 372503.949 41...",1714,6758
3,E02000987,Bolton 004,"POLYGON ((363078.556 411480.529, 363041.920 41...",1251,5891
4,E02000988,Bolton 005,"POLYGON ((371044.136 412457.779, 371567.570 41...",2176,6749
...,...,...,...,...,...
5803,E02006922,Colchester 022,"MULTIPOLYGON (((601481.105 224815.326, 601485....",2699,11342
5804,E02006926,Thurrock 020,"POLYGON ((561100.600 178929.667, 560928.979 17...",2342,5982
5805,E02006932,Liverpool 060,"POLYGON ((335757.632 390987.474, 335739.219 39...",511,18899
5806,E02006933,Liverpool 061,"POLYGON ((335096.788 389638.891, 334715.024 38...",654,8174


In [9]:
MSOA_excludeLondon_base.to_file("MSOA_Boundary_with_population_base.gpkg", driver="GPKG")